<a href="https://colab.research.google.com/github/ki7348/Algorithm/blob/main/wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [ ]:
import csv
wine_path = "./winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
wineq_numpy

In [ ]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

In [ ]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

In [ ]:
data = wineq[:, :-1] 

data, data.shape

In [ ]:
target = wineq[:, -1] 

target, target.shape

In [ ]:
target = wineq[:, -1].long()

target

In [ ]:
target_onehot = torch.zeros(target.shape[0],10)

target = target_onehot.scatter_(1, target.unsqueeze(1),1.0)

target

In [ ]:
n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices

In [ ]:
target_t = target[train_indices]
target_v = target[val_indices]

data_t = data[train_indices]
data_v = data[val_indices]

In [ ]:
loss_val_data = [] 
loss_train_data = [] 

def training_loop(n_epochs, optimizer, model, loss_fn, data_t, data_v,
                  target_t, target_v):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(data_t) # <1>
        loss_train = loss_fn(t_p_train, target_t)
        t_p_val = model(data_v) 
        loss_val = loss_fn(t_p_val, target_v)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        loss_val_data.append(loss_val.item())
        loss_train_data.append(loss_train.item())
        
        if epoch < 3 or epoch % 1000 == 0:
          print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

linear_model = nn.Linear(11, 10)
learning_rate = 1e-4
optimizer = optim.SGD(linear_model.parameters(), lr=learning_rate)

training_loop(
    n_epochs = 10000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), 
    data_t = data_t,
    data_v = data_v, 
    target_t = target_t,
    target_v = target_v)

In [ ]:
n_epochs = 10000
plt.title('train_result');
x = range(n_epochs)
y = loss_train_data
plt.xlim(5000,10000)
plt.ylim(0.085,0.12)
plt.plot(x,y)

y = loss_val_data
plt.plot(x,y)
plt.show()